In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import pandas as pd
from pyspark.ml.feature import VectorAssembler 
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql.functions import rand
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [2]:
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "2g")
    .config("spark.executer.memory", "4g")
    .getOrCreate()
)

22/09/24 18:02:56 WARN Utils: Your hostname, Luo resolves to a loopback address: 127.0.1.1; using 172.29.22.132 instead (on interface eth0)
22/09/24 18:02:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/24 18:02:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
sdf = spark.read.parquet('../data/curated/train_data/')

In [4]:
sdf.printSchema()

root
 |-- merchant_abn: long (nullable = true)
 |-- total_num_consumer: long (nullable = true)
 |-- avg_dollar_value: double (nullable = true)
 |-- total_num_transaction: long (nullable = true)
 |-- mean_income: double (nullable = true)
 |-- revenue_level: string (nullable = true)
 |-- total_revenue: double (nullable = true)
 |-- total_num_postcode: long (nullable = true)
 |-- tag: string (nullable = true)
 |-- y_total_num_consumer: long (nullable = true)
 |-- y_total_revenue: double (nullable = true)
 |-- y_total_num_transaction: long (nullable = true)



In [5]:
indexed_features = ['revenue_level', 'tag']
# We give all values in non-numeric features an index in order to make it ordinal or one-hot encoded
indexers =[]
for col in indexed_features:
  indexers.append(StringIndexer(inputCol=col, outputCol = col+"_index"))

pipeline = Pipeline(stages=indexers)
indexed_sdf = pipeline.fit(sdf).transform(sdf)

In [6]:
indexed_sdf

merchant_abn,total_num_consumer,avg_dollar_value,total_num_transaction,mean_income,revenue_level,total_revenue,total_num_postcode,tag,y_total_num_consumer,y_total_revenue,y_total_num_transaction,revenue_level_index,tag_index
10187291046,87,111.08408713922158,87,61060.0459770115,b,31795.597893195016,87,watch,99,41683.211213258364,100,1.0,9.0
10255988167,218,389.5552654520502,218,63146.619266055044,b,366867.5813701302,211,computer,235,378005.1467314967,236,1.0,0.0
10264435225,1238,114.10783402533235,1272,62006.311320754714,c,346896.9592900661,1018,watch,1519,435003.6795629895,1566,2.0,9.0
10364012396,4,276.08689369891994,4,81123.75,b,4008.7818228908673,4,music,16,19636.790814026932,16,1.0,11.0
10385250025,185,474.1675409676319,185,63441.41081081081,a,485974.30939143547,182,computer,190,494288.86012436583,191,0.0,0.0
10430380319,38,359.1108847340083,38,60029.76315789474,b,67821.67882777525,38,motor,53,88318.04454751333,53,1.0,17.0
10441711491,1,9734.857620793187,1,57015.0,a,56170.12828629902,1,motor,null,null,null,0.0,17.0
10462560289,405,37.94568323793217,413,64556.179176755446,c,46231.12392021321,366,gift,513,56043.0017308923,519,2.0,4.0
10463252268,22,464.0964976850653,22,60070.77272727273,a,67488.91405656068,22,artist supply,26,78474.65405470507,26,0.0,2.0
10530696903,174,429.6262128784364,175,61263.58857142857,a,485692.4365271367,167,books,225,599049.1086622305,225,0.0,12.0


In [7]:
categorical_features =  ["tag_index", "revenue_level_index"]
# one-hot-encoding the numeric indices
ohe = []
for f in categorical_features:
  ohe.append(OneHotEncoder(inputCol=f, outputCol=f+"OHE"))

pipeline = Pipeline(stages=ohe)
encoded_sdf = pipeline.fit(indexed_sdf).transform(indexed_sdf)

In [8]:
encoded_sdf.printSchema()

root
 |-- merchant_abn: long (nullable = true)
 |-- total_num_consumer: long (nullable = true)
 |-- avg_dollar_value: double (nullable = true)
 |-- total_num_transaction: long (nullable = true)
 |-- mean_income: double (nullable = true)
 |-- revenue_level: string (nullable = true)
 |-- total_revenue: double (nullable = true)
 |-- total_num_postcode: long (nullable = true)
 |-- tag: string (nullable = true)
 |-- y_total_num_consumer: long (nullable = true)
 |-- y_total_revenue: double (nullable = true)
 |-- y_total_num_transaction: long (nullable = true)
 |-- revenue_level_index: double (nullable = false)
 |-- tag_index: double (nullable = false)
 |-- tag_indexOHE: vector (nullable = true)
 |-- revenue_level_indexOHE: vector (nullable = true)



In [9]:
features = ['total_num_consumer', 'avg_dollar_value', 'total_num_transaction', 'mean_income', 'revenue_level_indexOHE', \
    'total_revenue', 'total_num_postcode', 'tag_indexOHE']
assembler = VectorAssembler(inputCols=features ,outputCol='features')
vectorised_training_set = assembler.transform(encoded_sdf)

In [10]:
vectorised_training_set = vectorised_training_set.select('features', 'y_total_revenue')

In [11]:
vectorised_training_set

features,y_total_revenue
"(33,[0,1,2,3,5,8,...",41683.211213258364
"(33,[0,1,2,3,5,8,...",378005.1467314967
"(33,[0,1,2,3,6,8,...",435003.6795629895
"(33,[0,1,2,3,5,8,...",19636.790814026932
"(33,[0,1,2,3,4,8,...",494288.86012436583
"(33,[0,1,2,3,5,8,...",88318.04454751333
"(33,[0,1,2,3,4,8,...",null
"(33,[0,1,2,3,6,8,...",56043.0017308923
"(33,[0,1,2,3,4,8,...",78474.65405470507
"(33,[0,1,2,3,4,8,...",599049.1086622305


In [12]:
train,test = vectorised_training_set.randomSplit([0.7,0.3])

In [13]:
train.count(), test.count()

(2751, 1202)

In [14]:
lr = LogisticRegression(labelCol='y_total_revenue')

22/09/24 22:32:47 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 6344078 ms exceeds timeout 120000 ms
22/09/24 22:32:47 WARN SparkContext: Killing executors is not supported by current scheduler.


In [26]:
train.where(F.col('y_total_revenue'))

TypeError: 'Column' object is not callable

In [27]:
fitted_model = lr.fit(train)
#fitted_model.setFeaturesCol("features")
#fitted_model.setPredictionCol("prediction")

22/09/24 22:41:03 ERROR Executor: Exception in task 1.0 in stage 23.0 (TID 24)
scala.MatchError: [null,1.0,(33,[0,1,2,3,4,8,9],[12.0,18309.168774046513,12.0,58310.25,1.0,1333639.8912172737,12.0])] (of class org.apache.spark.sql.catalyst.expressions.GenericRowWithSchema)
	at org.apache.spark.ml.PredictorParams.$anonfun$extractInstances$1(Predictor.scala:81)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:461)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1431)
	at scala.collection.TraversableOnce.foldLeft(TraversableOnce.scala:199)
	at scala.collection.TraversableOnce.foldLeft$(TraversableOnce.scala:192)
	at scala.collection.AbstractIterator.foldLeft(Iterator.scala:1431)
	at scala.collection.TraversableOnce.aggregate(TraversableOnce.scala:260)
	at scala.collection.TraversableOnce.aggregate$(TraversableOnce.scala:260)
	at scala.collection.Abstract

Py4JJavaError: An error occurred while calling o287.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 23.0 failed 1 times, most recent failure: Lost task 1.0 in stage 23.0 (TID 24) (172.29.22.132 executor driver): scala.MatchError: [null,1.0,(33,[0,1,2,3,4,8,9],[12.0,18309.168774046513,12.0,58310.25,1.0,1333639.8912172737,12.0])] (of class org.apache.spark.sql.catalyst.expressions.GenericRowWithSchema)
	at org.apache.spark.ml.PredictorParams.$anonfun$extractInstances$1(Predictor.scala:81)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:461)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1431)
	at scala.collection.TraversableOnce.foldLeft(TraversableOnce.scala:199)
	at scala.collection.TraversableOnce.foldLeft$(TraversableOnce.scala:192)
	at scala.collection.AbstractIterator.foldLeft(Iterator.scala:1431)
	at